**Tabla de contenido Capitulo 1**

- [Hello Transformers](#Hello-Transformers)
    - [A Tour of Transformer Applications](#A-Tour-of-Transformer-Applications)
    - [Text Classification](#Text-Classification)

# Hello Transformers

## A Tour of Transformer Applications


Cada tarea de PNL comienza con un fragmento de texto, como los siguientes comentarios inventados de los clientes sobre un determinado sitio en línea orden:

In [3]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

Dependiendo de su aplicación, el texto con el que esté trabajando podría ser un contrato legal, una descripción del producto o algo completamente diferente. En el caso de los comentarios de los clientes, probablemente le gustaría saber si los comentarios son `positivos` o `negativos`.

Esta tarea se denomina `análisis de sentimientos` y forma parte del tema más amplio de clasificación de textos que exploraremos en el capítulo 2. Por ahora, echemos un `vistazo` a lo que `se necesita` para extraer el sentimiento de nuestro texto usando Transformers.


## Text Classification

En Transformers, creamos una instancia de `pipeline` llamando a la función `pipeline()` y proporcionando el nombre de la tarea en la que estamos interesados:

In [1]:
# pip install ipywidgets
from transformers import pipeline
classifier = pipeline("text-classification", framework="pt")  # framework="pt" fuerza PyTorch

2025-06-12 17:27:18.028207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749767238.043650   47499 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749767238.048669   47499 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749767238.061354   47499 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749767238.061370   47499 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749767238.061372   47499 computation_placer.cc:177] computation placer alr

¡Ahora que tenemos nuestro pipeline, generemos algunas predicciones! Cada pipeline toma una cadena de texto (o una lista de cadenas) como entrada y devuelve un lista de predicciones. Cada predicción es un diccionario de Python, por lo que podemos usar Pandas para mostrarlos los resultados.

In [4]:
import pandas as pd
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546
